In [1]:
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm

In [3]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.sequence import pad_sequences

In [4]:
import tensorflow as tf
def file_to_sentence_list(file_path):
	with open(file_path, 'r') as file:
		text = file.read()

	# Splitting the text into sentences using
	# delimiters like '.', '?', and '!'
	sentences = [sentence.strip() for sentence in re.split(
		r'(?<=[.!?])\s+', text) if sentence.strip()]

	return sentences

file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
	input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [5]:
model = Sequential()
model.add(Embedding(total_words,10,input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=100)

Epoch 1/100
51/51 [==============================] - 7s 59ms/step - loss: 6.2175 - accuracy: 0.0528
Epoch 2/100
51/51 [==============================] - 3s 57ms/step - loss: 5.7895 - accuracy: 0.0510
Epoch 3/100
51/51 [==============================] - 3s 60ms/step - loss: 5.7314 - accuracy: 0.0528
Epoch 4/100
51/51 [==============================] - 5s 91ms/step - loss: 5.7091 - accuracy: 0.0553
Epoch 5/100
51/51 [==============================] - 3s 62ms/step - loss: 5.6747 - accuracy: 0.0565
Epoch 6/100
51/51 [==============================] - 3s 56ms/step - loss: 5.6315 - accuracy: 0.0541
Epoch 7/100
51/51 [==============================] - 3s 59ms/step - loss: 5.5822 - accuracy: 0.0565
Epoch 8/100
51/51 [==============================] - 4s 88ms/step - loss: 5.5305 - accuracy: 0.0553
Epoch 9/100
51/51 [==============================] - 4s 72ms/step - loss: 5.4796 - accuracy: 0.0590
Epoch 10/100
51/51 [==============================] - 3s 54ms/step - loss: 5.4290 - accuracy: 0.0706

In [6]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 [==============================] - 0s 50ms/step
Next predicted words: Pizza have different  frango com catupiry is a
